In [1]:
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_val, y_val) = keras.datasets.fashion_mnist.load_data()   #built in dataset in tensorflow

4431872/4422102 [==============================] - 1s 0us/step


### Data preprocessing


In [15]:
def preprocess(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)

  return x, y

def create_dataset(xs, ys, n_classes=10):
  ys = tf.one_hot(ys, depth=n_classes)
  return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(128)


### Creating neural network

In [16]:
train_dataset = create_dataset(x_train, y_train)
val_dataset = create_dataset(x_val, y_val)

2022-03-05 13:19:04.836269: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
def custom_l2_regularizer(weights):
    return tf.reduce_sum(0.02 * tf.square(weights))

### Building the neural network

In [72]:

model = keras.Sequential([
    keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=192, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])

### Training the model 


In [73]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_dataset.repeat(), 
    epochs=20, 
    steps_per_epoch=500,
    validation_data=val_dataset.repeat(), 
    validation_steps=2
)

Epoch 1/20
500/500 [==============================] - 6s 9ms/step - loss: 0.5001 - accuracy: 0.8219 - val_loss: 0.3845 - val_accuracy: 0.8633
Epoch 2/20
500/500 [==============================] - 4s 9ms/step - loss: 0.3564 - accuracy: 0.8692 - val_loss: 0.3810 - val_accuracy: 0.8477
Epoch 3/20
500/500 [==============================] - 5s 10ms/step - loss: 0.3203 - accuracy: 0.8833 - val_loss: 0.3355 - val_accuracy: 0.8906
Epoch 4/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2969 - accuracy: 0.8909 - val_loss: 0.3744 - val_accuracy: 0.8711
Epoch 5/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2808 - accuracy: 0.8955 - val_loss: 0.3401 - val_accuracy: 0.8789
Epoch 6/20
500/500 [==============================] - 4s 9ms/step - loss: 0.2615 - accuracy: 0.9009 - val_loss: 0.3600 - val_accuracy: 0.8711
Epoch 7/20
500/500 [==============================] - 5s 10ms/step - loss: 0.2510 - accuracy: 0.9063 - val_loss: 0.3577 - val_accuracy: 0.8594
Ep

### Testing the neural network

In [74]:
predictions = model.predict(val_dataset)

In [75]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_9 (Reshape)         (None, 784)               0         
                                                                 
 dense_48 (Dense)            (None, 256)               200960    
                                                                 
 dense_49 (Dense)            (None, 192)               49344     
                                                                 
 dense_50 (Dense)            (None, 128)               24704     
                                                                 
 dense_51 (Dense)            (None, 10)                1290      
                                                                 
Total params: 276,298
Trainable params: 276,298
Non-trainable params: 0
_________________________________________________________________


In [77]:
!mkdir -p saved_model
model.save('saved_model/')

INFO:tensorflow:Assets written to: saved_model/assets


In [79]:
loss, acc = model.evaluate(val_dataset, verbose=2)
print('model, accuracy: {:5.2f}%'.format(100 * acc))

79/79 - 1s - loss: 0.3672 - accuracy: 0.8950 - 621ms/epoch - 8ms/step
model, accuracy: 89.50%
